This [Jupyter notebook](https://jupyter.org/)
is meant to be viewed in "slide mode".

If you want to view the slides locally
(with notebook version < 7),
you'll need to install [RISE](https://rise.readthedocs.io/).
With notebook version 7 and above,
and with JupyterLab,
you can try the [`jupyterlab_rise`](https://github.com/jupyterlab-contrib/rise) extension.

But it's probably simpler to just visit the
[online version](https://nbviewer.org/format/slides/github/mgeier/defense/blob/main/presentation.ipynb).

# Describing Three-Dimensional Movements in an Audio Scene Authoring Format

Matthias Geier

Thesis:
[PDF](https://github.com/mgeier/diss/releases/download/initial-submission/Geier_Diss.pdf),
[LaTeX sources](https://github.com/mgeier/diss)  
Slides: https://github.com/mgeier/defense

## Table of Contents

* Spatial Audio From the Beginning
  * object-based audio reproduction
* Movement and Time in Existing Formats
  * VRML, X3D, MPEG-4 AudioBIFS, Audio3D, XML3DAUDIO, ADM, SpatDIF, Spat-SDIF, TASCAR, SMIL
* Development of a Scene Authoring Format: ASDF
* Implementation and Integration of an ASDF Library
* Conclusion and Future Work
* Appendix A: ASDF specification & documentation
* Appendix B: Splines
  * Euclidean
  * rotation

## Spatial Audio

* binaural
* WFS
* Ambisonics
* VBAP
* ...

problem:

* many output channels
* heterogeneous setups

solution:

* object-based audio reproduction

### Object-Based Audio Reproduction

* sound sources (mono signals)
* groups of sound sources
* observer position
* three-dimensional movement & trajectories
  * translation
  * rotation
  * movements in local coordinate systems, which are moving themselves
* dynamic volume control

## Audio Scene Description Format (ASDF)

https://AudioSceneDescriptionFormat.readthedocs.io/

```xml
<asdf version="0.4">
  <clip file="audio/ukewave.ogg" pos="1 2" />
</asdf>
```

### Time Containers

`<par>` and `<seq>` inspired by the [SMIL](https://www.w3.org/TR/SMIL/) format:

```xml
<asdf version="0.4">
  <par>
    <clip file="audio/ukewave.ogg" pos="0 2" />
    <seq>
      <clip file="audio/marimba.ogg">
        <channel pos="-1 2" />
        <channel pos="1 2" />
      </clip>
      <clip file="audio/xmas.wav" pos="-1.5 0" />
    </seq>
  </par>
</asdf>
```

### Trajectories with Time and Speed

```xml
<asdf version="0.4">
  <par>
    <clip id="ukulele" file="audio/ukewave.ogg" />
    <transform apply-to="ukulele">
      <o pos="-2 -2" speed="0" />
      <o pos="-2 2" />
      <o pos="2 2" time="15" speed="0.5" />
      <o pos="2 -2" />
    </transform>
  </par>
</asdf>
```

### Nested Transforms

```xml
<asdf version="0.4">
  <par>
    <clip id="ukulele" file="audio/ukewave.ogg" pos="-2 -2" />
    <transform id="horizontal-movement" apply-to="ukulele" repeat="10">
      <o pos="2 4" />
      <o pos="0 2" />
      <o pos="2 0" />
      <o pos="4 2" />
      <o pos="closed" />
    </transform>
    <transform apply-to="horizontal-movement">
      <o rot="0" />
      <o rot="0 0 90" />
      <o rot="0 0 180" />
    </transform>
  </par>
</asdf>
```

### Mixed Transform Attributes

```xml
<asdf version="0.4">
  <par>
    <clip id="marimba" file="audio/marimba.ogg">
      <channel pos="-1 0" />
      <channel pos="1 0" />
    </clip>
    <transform apply-to="marimba">
      <o pos="0 -2" rot="-20" vol="1" />
      <o pos="0 0" time="1s" />
      <o vol="1" />
      <o rot="0" time="2s" />
      <o vol="0" />
      <o vol="1" time="65%" />
      <o pos="0 2" rot="20" vol="1"/>
    </transform>
  </par>
</asdf>
```

## Splines

Goal: scene author defines a few points in 3D space, a smooth curve is generated

further goals:

* don't stray too far from the control points
* changes in one place should only affect the immediate surroundings
* control timing, control speed

afterwards:

* same for rotation!

In [ ]:
%config InlineBackend.figure_formats = ['svg']

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import splines
from euclidean_helper import plot_spline_1d, plot_slopes_1d, grid_lines
from euclidean_helper import plot_spline_2d, plot_tangents_2d, plot_tangent_2d

### Example Trajectory

In [ ]:
vertices = [
    (-2, -1),
    (-2, 0),
    (-0.1, 1.5),
    (0.1, 1.5),
    (2, 0),
    (3, 1),
]

In [ ]:
def plot_constant_speed_spline(vertices, dots_per_second=4):
    plot_spline_2d(
        splines.UnitSpeedAdapter(splines.CatmullRom(vertices, alpha=0.5)),
        dots_per_second=dots_per_second)

In [ ]:
plot_constant_speed_spline(vertices)

* many systems use linear interpolation, but smooth movement is nicer
* cubic splines are a good trade-off between flexibility and simplicity

### Cubic Polynomial Curves (one-dimensional)

\begin{equation*}
\boldsymbol{p}_i(t) = \boldsymbol{d}_i t^3 + \boldsymbol{c}_i t^2 + \boldsymbol{b}_i t + \boldsymbol{a}_i
\end{equation*}

In [ ]:
def plot_slopes(s):
    grid = s.grid
    plot_slopes_1d(s.evaluate(grid, 1), s.evaluate(grid), grid, scale=2)

In [ ]:
s_x = splines.Monomial([[-1.5, 0, 1.5, 0]])
s_y = splines.Monomial([[2.75, -4.5, 0.75, 1]])

In [ ]:
plot_spline_1d(s_x); plot_slopes(s_x)
plot_spline_1d(s_y); plot_slopes(s_y)
plt.xlabel('t')
grid_lines([0, 1]); plt.ylim([-0.1, 1.5]);

### Cubic Polynomial Curves (two-dimensional)

\begin{equation*}
\boldsymbol{p}_i(t) = \boldsymbol{d}_i t^3 + \boldsymbol{c}_i t^2 + \boldsymbol{b}_i t + \boldsymbol{a}_i
\end{equation*}

In [ ]:
s1 = splines.Monomial([[[-1.5, 2.75], [0, -4.5], [1.5, 0.75], [0, 1]]])

In [ ]:
def plot_tangents(s):
    plot_tangents_2d(s.evaluate(s.grid, 1), s.evaluate(s.grid))

In [ ]:
plot_spline_2d(s1, chords=False)
plot_tangents(s1)

This also works for 3D and higher dimensions.

### Calculating Coefficients from Positions & Slopes

\begin{equation*}
\boldsymbol{p}_i(t) =
\left[\begin{matrix}t^{3} & t^{2} & t & 1\end{matrix}\right]
\left[\begin{matrix}2 & -2 & 1 & 1\\-3 & 3 & -2 & -1\\0 & 0 & 1 & 0\\1 & 0 & 0 & 0\end{matrix}\right]
\left[\begin{matrix}\boldsymbol{x}_{i}\\\boldsymbol{x}_{i+1}\\\boldsymbol{\dot{x}}_{i}\\\boldsymbol{\dot{x}}_{i+1}\end{matrix}\right]
\end{equation*}

### Smoothly Connecting Segments

* uniform Catmull–Rom splines
  (based on quadratic Lagrange interpolation and linear blending)
* between segments: incoming tangent equals outgoing tangent
* simple tangent equation:

  \begin{equation*}
  \boldsymbol{\dot{x}}_i = \frac{\boldsymbol{x}_{i+1} - \boldsymbol{x}_{i-1}}{2}
  \end{equation*}

In [ ]:
def plot_cr_tangent():
    points1 = [
        (-1, -0.5),
        (0, 2.3),
        (1, 1),
        (4, 1.3),
        (3.8, -0.2),
        (2.5, 0.1),
    ]
    idx = 2
    s1 = splines.CatmullRom(points1, endconditions='closed')
    plot_spline_2d(s1)
    plot_tangent_2d(
        s1.evaluate(s1.grid[idx], 1),
        s1.evaluate(s1.grid[idx]), color='purple')
    plt.plot(
        *s1.evaluate([s1.grid[idx - 1], s1.grid[idx + 1]]).T,
        '--', color='purple', linewidth=2)

In [ ]:
plot_cr_tangent()

### Cusps, Self-Intersections

* uniform parameter intervals (for each segment from 0 to 1)

In [ ]:
plot_spline_2d(splines.CatmullRom(vertices))

### Non-Uniform Splines

* arbitrary parameter interval for each segment

* tangent equation for uniform Catmull–Rom spline:

  \begin{equation*}
  \boldsymbol{\dot{x}}_i = \frac{\boldsymbol{x}_{i+1} - \boldsymbol{x}_{i-1}}{2}
  \end{equation*}
  
* tangent equation for non-uniform Catmull–Rom spline:

  \begin{equation*}
  \boldsymbol{\dot{x}}_i =
  \frac{
  (t_{i+1} - t_i) (\boldsymbol{x}_i - \boldsymbol{x}_{i-1})
  }{
  (t_i - t_{i-1})(t_{i+1} - t_{i-1})
  }
  +
  \frac{
  (t_i - t_{i-1}) (\boldsymbol{x}_{i+1} - \boldsymbol{x}_i)
  }{
  (t_{i+1} - t_i)(t_{i+1} - t_{i-1})
  }
  \end{equation*}

### Centripetal Parameterization

* parameter interval is chosen to be the Euclidean distance between start and end of segment

In [ ]:
plot_spline_2d(splines.CatmullRom(vertices, alpha=0.5))

### Arc-Length Parameterization

* so far: good shape, but the timing is not really controllable.

* therefore: re-parameterization
  (creating a new parameter that's then mapped back to the original parameter)

  \begin{equation*}
  \boldsymbol{x}_\text{arc}(s) = \boldsymbol{x}(t(s))
  \end{equation*}

* arc-length parameterization $\to$ unit speed

  \begin{equation*}
  s(t) = \int\limits_{t_0}^t \left| \frac{d}{d\tau}\boldsymbol{x}(\tau) \right| d\tau
  \end{equation*}

* no analytic solution

* numerical integration, then find the inverse function with the bisection method

### Arc-Length Parameterized Centripetal Catmull–Rom spline

In [ ]:
plot_spline_2d(splines.UnitSpeedAdapter(splines.CatmullRom(vertices, alpha=0.5)))

### More Re-Parameterization

* constant speed is a good default, but more control would be good
* therefore: another level of re-parameterization
  based on [monotone splines](https://splines.readthedocs.io/en/0.3.1/euclidean/piecewise-monotone.html)
* time instants can be specified for any vertex
* speed (in m/s) can be specified (but with limits, to ensure monotonicity)

In [ ]:
plot_spline_2d(
    splines.NewGridAdapter(
        splines.UnitSpeedAdapter(splines.CatmullRom(vertices, alpha=0.5)),
        [0, None, None, None, 3, 5]))

## Rotation Splines

Goal: use same techniques as in the Euclidean case:

* cubic splines (Catmull–Rom/Kochanek–Bartels)
* non-uniform, centripetal
* constant speed re-parameterization
* monotone spline re-parameterization

### (Unit) Quaternions

quaternions:

* 4-dimensional space
* 4 components, "scalar" and "vector" parts:
  \begin{equation*}
  q = (w, \vec{v}) = (w, (x, y, z))
  \end{equation*}
* multiplication is non-commutative

unit quaternions:

* all quaternions with length 1 $\to$ unit hypersphere $S^3$
* can be used to describe rotations (with "double cover")

### Unit Quaternions as Rotations

\begin{equation*}
q = \left(\cos \frac{\alpha}{2}, \vec{n} \sin \frac{\alpha}{2}\right)
\end{equation*}

In [ ]:
from quaternion_helper import angles2quat, plot_rotation, animate_rotations, display_animation

In [ ]:
identity = angles2quat(0, 0, 0)
identity

In [ ]:
a = angles2quat(90, 0, 0)
b = angles2quat(0, 35, 0)
c = angles2quat(0, 0, 45)

In [ ]:
plot_rotation({
    'identity = 1': identity,
    '$a$': a,
    '$b$': b,
    '$c$': c,
});

## Quaternion Multiplication

non-commutative:

\begin{equation*}
q_m q_n \ne q_n q_m
\end{equation*}

In [ ]:
plot_rotation({'$ab$': a * b, '$ba$': b * a});

## Quaternion Multiplication

associative:

\begin{equation*}
(q_1 q_2) q_3 = q_1 (q_2 q_3)
\end{equation*}

In [ ]:
plot_rotation({'$(bc)a$': (b * c) * a, '$b(ca)$': b * (c * a)});

## Inverse

written as: $q^{-1}$

In [ ]:
plot_rotation({'$b$': b, '$b^{-1}$': b.inverse()});

\begin{equation*}
q q^{-1} = q^{-1} q = \boldsymbol{1}
\end{equation*}

for unit quaternions: $q^{-1} = \overline{q}$

\begin{equation*}
\overline{q}
= \left(w, -\vec{v}\right)
= \left(\cos \frac{\alpha}{2}, -\vec{n} \sin \frac{\alpha}{2}\right)
= \left(\cos \frac{-\alpha}{2}, \vec{n} \sin \frac{-\alpha}{2}\right)
\end{equation*}

## Exponentiation (integer exponents)

In [ ]:
plot_rotation({
    '$a^0 = 1$': a**0,
    '$a^1 = a$': a**1,
    '$a^2 = aa$': a**2,
    '$a^3 = aaa$': a**3,
});

## Exponentiation (real exponents)

In [ ]:
plot_rotation({
    '$a^1 = a$': a**1,
    '$a^{0.5}$': a**0.5,
    '$a^0 = 1$': a**0,
    '$a^{-0.5}$': a**-0.5,
});

\begin{equation*}
q^k = \left(\cos \frac{k\alpha}{2}, \vec{n} \sin \frac{k\alpha}{2}\right)
\end{equation*}

## Negation

In [ ]:
plot_rotation({'$c$': c, '$-c$': -c});

\begin{equation*}
-q
= \left(-w, -\vec{v}\right)
= \left(
\cos \frac{\alpha + 2 \pi}{2},
\vec{n} \sin \frac{\alpha + 2 \pi}{2}
\right)
\end{equation*}

### How To Interpolate Unit Quaternions?

* polynomials in a curved space?

* use De Casteljau's algorithm with [spherical linear interpolation (Slerp)](https://splines.readthedocs.io/en/0.3.1/rotation/slerp.html)
  * see the following slides

  \begin{equation*}
  \operatorname{Slerp}(q_0, q_1; t) = \left(q_1 {q_0}^{-1}\right)^t \, q_0
  \end{equation*}

### Slerp Animation

In [ ]:
from splines.quaternion import slerp

In [ ]:
q1 = angles2quat(45, -20, -60)
q2 = angles2quat(-45, 20, 30)

In [ ]:
ani_times = np.linspace(0, 1, 100)
slerp_ani = animate_rotations({
    'slerp(q1, q2)': slerp(q1, q2, ani_times),
    'slerp(q1, -q2)': slerp(q1, -q2, ani_times),
})

In [ ]:
display_animation(slerp_ani, default_mode='reflect')

### Using Lines to Construct Curves: De Casteljau's Algorithm

* control points at $\frac{1}{3}$ of the tangent vectors
* $\boldsymbol{x}_0$, $\boldsymbol{\tilde{x}}_0$,
  $\boldsymbol{\tilde{x}}_1$, $\boldsymbol{x}_1$:
  * 4 control points of a *cubic Bézier curve*

In [ ]:
def plot_control_points(s):
    a = s.evaluate(0) + s.evaluate(0, 1) / 3
    b = s.evaluate(1) - s.evaluate(1, 1) / 3
    plt.scatter(*np.column_stack([a, b]), marker='X', color='black')
    plt.annotate(r'$\boldsymbol{x}_0$', s.evaluate(0) + [0, 0.1])
    plt.annotate(r'$\tilde{\boldsymbol{x}}_0$', a + [0, 0.1])
    plt.annotate(r'$\tilde{\boldsymbol{x}}_1$', b + [0, 0.1])
    plt.annotate(r'$\boldsymbol{x}_1$', s.evaluate(1) + [0, 0.1]);

In [ ]:
plot_spline_2d(s1, chords=False)
plot_tangents(s1)
plot_control_points(s1)

### De Casteljau's Algorithm, Animated

In [ ]:
from casteljau import create_animation

def show_casteljau_animation(points, frames=30, interval=200):
    ani = create_animation(points, frames=frames, interval=interval)
    display({
        'text/html': ani.to_jshtml(default_mode='reflect'),
    }, raw=True)
    plt.close()  # avoid spurious figure display

In [ ]:
show_casteljau_animation([(0, 1), (0.5, 1.25), (1, 0), (0, 0)])

For rotations: replace linear interpolations with Slerp!

### Joining Tangents

Goal: find the missing Bézier control points
using Catmull–Rom tangents.

Based on the non-uniform
[equations for the Euclidean case](https://splines.readthedocs.io/en/0.3.1/euclidean/catmull-rom-non-uniform.html#Using-Non-Uniform-Bézier-Segments),
we are trying to come up with
[equations for rotations using unit quaternions](https://splines.readthedocs.io/en/0.3.1/rotation/catmull-rom-non-uniform.html)
(see next slide).

\begin{align*}
\boldsymbol{v}_i &= \frac{
\boldsymbol{x}_{i+1} - \boldsymbol{x}_i
}{
t_{i+1} - t_i
}
\\
\boldsymbol{\dot{x}}_i
&= \frac{
(t_{i+1} - t_i) \, \boldsymbol{v}_{i-1} + (t_i - t_{i-1}) \, \boldsymbol{v}_i
}{
t_{i+1} - t_{i-1}
}
\\
\boldsymbol{\tilde{x}}_i^{(+)}
&= \boldsymbol{x}_i + \frac{(t_{i+1} - t_i) \, \boldsymbol{\dot{x}}_i}{3}
\\
\boldsymbol{\tilde{x}}_i^{(-)}
&= \boldsymbol{x}_i - \frac{(t_i - t_{i-1}) \, \boldsymbol{\dot{x}}_i}{3}
\end{align*}

----

\begin{align*}
\vec{\rho}_{i} &= \frac{\ln(q_{i+1} {q_i}^{-1})}{t_{i+1} - t_i}
\\
\vec{\omega}_i &=
\frac{
(t_{i+1} - t_i) \, \vec{\rho}_{i-1} +
(t_i - t_{i-1}) \, \vec{\rho}_{i}
}{
t_{i+1} - t_{i-1}
}
\\
\tilde{q}_i^{(+)}
&=
\exp\left(\frac{t_{i+1} - t_i}{3} \, \vec{\omega}_i\right) \, q_i
\\
\tilde{q}_i^{(-)}
&=
\exp\left(\frac{t_i - t_{i-1}}{3} \, \vec{\omega}_i\right)^{-1} \, q_i
\end{align*}

### Rotation Splines, Summary

Goal: use same techniques as in the Euclidean case:

* cubic splines (Catmull–Rom/Kochanek–Bartels)
  * De Casteljaus's algorithm with Slerp
* non-uniform, centripetal
  * similar to Euclidean case
* constant speed re-parameterization
  * de Casteljau's algorithm provides first derivative (angular velocity)
  * `splines.UnitSpeedAdapter`
* monotone spline re-parameterization
  * `splines.NewGridAdapter`

## Integration of the ASDF Library

* stand-alone spatialization app
  * SoundScape Renderer
  * integration with JACK transport
  
* plugin for multimedia framework
  * Pure Data (https://puredata.info/)
  * `asdf~`

### Software and Documentation

* ASDF spec with examples: https://github.com/AudioSceneDescriptionFormat/asdf
* ASDF library: https://github.com/AudioSceneDescriptionFormat/asdf-rust
  * implemented in Rust (providing C language bindings)
  * incl. external for Pure Data: `asdf~` (implemented in C)
* Splines documentation: https://splines.readthedocs.io/
  * incl. Python module: `splines`
* Splines library: https://github.com/AudioSceneDescriptionFormat/asdfspline-rust
  * implemented in Rust (providing Python language bindings)
* SoundScape Renderer (incl. ASDF support): https://github.com/SoundScapeRenderer/ssr
* Thesis: https://github.com/mgeier/diss
* Slides: https://github.com/mgeier/defense

Everything is Open Source!